# Using the IBM FHIR Server Location Search in Jupyter

**Author**: [Paul Bastide](mailto:pbastide@us.ibm.com)

The IBM FHIR Server supports FHIR Search with the SearchParameter `near` on the Location API. 

This notebook shows how to use the IBM FHIR Server examples with Location Search and use folium to render their plots.

More advanced demonstrations can be generated from this location data which reference these Locations, and are retrieved using [FHIR Search - Reference](https://www.hl7.org/fhir/search.html#reference).

In [6]:
# This snippet install geojson and geopandas to faciliate rendering in folium. 

# geojson to process the GIS data
!pip3 install geojson

#http://geopandas.org/
!pip3 install git+git://github.com/geopandas/geopandas.git

  Cloning git://github.com/geopandas/geopandas.git to /private/var/folders/07/sw3n5r3170q202d5j4tx8fhw0000gn/T/pip-req-build-q7wcs69j
  Running command git clone -q git://github.com/geopandas/geopandas.git /private/var/folders/07/sw3n5r3170q202d5j4tx8fhw0000gn/T/pip-req-build-q7wcs69j
  Created wheel for geopandas: filename=geopandas-0.8.0+63.gcdb4282-py2.py3-none-any.whl size=971092 sha256=66b8bd5ff1fe762b87d9cc8d0c95c1e56c75438884982f99ab0b539c8251311c
  Stored in directory: /private/var/folders/07/sw3n5r3170q202d5j4tx8fhw0000gn/T/pip-ephem-wheel-cache-whvym0hp/wheels/cf/3e/0b/6475054094c2b1ea054158ac1fdcf749fb92f5b512377e4cf8
Successfully built geopandas


In [37]:
# The following imports are used in the analysis of the location data:

# imported to retrieve any secondary data
import requests

# import pandas to process the data (and json_normalize to use the geojson as a plain dataframe)
import pandas as pd
import pandas.json_normalize
import numpy as np

# Import to control the output of the choropleth map
import matplotlib.cm as cm
import matplotlib.colors as colors

# standard matplot lib 
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.dates as mdates

# Import Folium to visualize the map
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster

# Import the GeoPandas utility to process / use establish the polygon area
import geopandas 

# figures out the neighborhoods for each coordinate. 
from shapely.geometry import Point 
#(switched to geojson point to more precisely identify location)
#from geojson import Point

# Imports the Clustering Utilities
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import json
from pandas.io.json import json_normalize

# used to process fhir-examples
import zipfile, io
import urllib3

print("All imports complete")

ModuleNotFoundError: No module named 'pandas.json_normalize'

# Download the fhir-examples
The following downloads the fhir-examples module.

In [10]:
# The URl for the most recent fhir-examples module.
fhir_examples_jar_url='https://dl.bintray.com/ibm-watson-health/ibm-fhir-server-releases/com/ibm/fhir/fhir-examples/4.5.0/fhir-examples-4.5.0.jar'

r = requests.get(fhir_examples_jar_url)
z = zipfile.ZipFile(io.BytesIO(r.content))

In [28]:
# Authenticate
from requests.auth import HTTPBasicAuth
httpAuth = HTTPBasicAuth('fhiruser', 'change-password')

# Get the Location using Requests
headers = {
    'Content-Type': 'application/json'
}

# We choose to disable warnings, in production, please don't
urllib3.disable_warnings()

In [31]:
# At this point we have z as a zip file and we can selectively process the zip file. 
# this is kind of cool as we can go ahead and send these files right to the IBM FHIR Server. 
entries = z.namelist()
for entry in entries:  
    #print(entry)
    if entry.startswith('json/ibm/bulk-data/location/'):
        f = z.open(entry);
        content = f.read()
        r = requests.post('https://localhost:9443/fhir-server/api/v4/Location', 
                    data=content,
                    headers=headers,  
                    auth=httpAuth,
                    verify=False)
        print('Done uploading - ' + entry)

Done uploading - json/ibm/bulk-data/location/
Done uploading - json/ibm/bulk-data/location/synthetic-mass-103.json
Done uploading - json/ibm/bulk-data/location/synthetic-mass-104.json
Done uploading - json/ibm/bulk-data/location/synthetic-mass-31.json
Done uploading - json/ibm/bulk-data/location/synthetic-mass-37.json
Done uploading - json/ibm/bulk-data/location/synthetic-mass-43.json
Done uploading - json/ibm/bulk-data/location/synthetic-mass-44.json
Done uploading - json/ibm/bulk-data/location/synthetic-mass-8.json
Done uploading - json/ibm/bulk-data/location/synthetic-mass-118.json
Done uploading - json/ibm/bulk-data/location/synthetic-mass-95.json
Done uploading - json/ibm/bulk-data/location/synthetic-mass-99.json
Done uploading - json/ibm/bulk-data/location/synthetic-mass-85.json
Done uploading - json/ibm/bulk-data/location/synthetic-mass-63.json
Done uploading - json/ibm/bulk-data/location/synthetic-mass-113.json
Done uploading - json/ibm/bulk-data/location/synthetic-mass-34.json

Done uploading - json/ibm/bulk-data/location/synthetic-mass-28.json
Done uploading - json/ibm/bulk-data/location/synthetic-mass-59.json
Done uploading - json/ibm/bulk-data/location/synthetic-mass-27.json
Done uploading - json/ibm/bulk-data/location/synthetic-mass-75.json
Done uploading - json/ibm/bulk-data/location/synthetic-mass-41.json
Done uploading - json/ibm/bulk-data/location/synthetic-mass-67.json
Done uploading - json/ibm/bulk-data/location/synthetic-mass-62.json
Done uploading - json/ibm/bulk-data/location/synthetic-mass-89.json


# Query the IBM FHIR Server with near

In [33]:
# Authenticate
from requests.auth import HTTPBasicAuth
httpAuth = HTTPBasicAuth('fhiruser', 'change-password')

In [45]:
# Get the Location using Requests
headers = {
    'Content-Type': 'application/json'
}

# Search within 10Km of Cambridge Massachusetts
queryParams = {
    'near': '42.373611|-71.110558|10|km',
    "_count" : 200
}

# Search Parameter = near=
resp_loc = requests.get('https://localhost:9443/fhir-server/api/v4/Location', 
                    headers=headers,  
                    params=queryParams,
                    auth=httpAuth,
                    verify=False)

In [46]:
# Convert to a list
response_loc_json = json.loads(resp_loc.text)

location_rows = []
for location in response_loc_json['entry']:
    row = pd.json_normalize(location)
    lat_inc = float(row['resource.position.latitude'])
    location_rows.append(row)
    
# If you are debugging... print(rows_list)

In [47]:
print(len(location_rows))

66


In [48]:
# zoom 15 is used to show the groupings
cambridge = [ 42.373611, -71.11000]
map_cambridge_locs_from_server = folium.Map(location=cambridge, zoom_start=10)

# Iterate through the Rows
for location_row in location_rows :
        # print(location_row)
        
        # Cast the values into the appropriate types as FOLIUM will die weirdly without it. 
        lat_inc = float(location_row['resource.position.latitude'])
        long_inc = float(location_row['resource.position.longitude'])
        name_inc = str(location_row['resource.name'])
        
        #print(lat_inc)
        #print(long_inc)
        #print(name_inc)
        
        label = folium.Popup(name_inc, parse_html=True)
        folium.CircleMarker(
            [lat_inc, long_inc],
            radius=5,
            popup=label,
            fill=True,
            fill_color='red',
            fill_opacity=0.7).add_to(map_cambridge_locs_from_server)
        
map_cambridge_locs_from_server

(C) Copyright IBM Corp. 2020

SPDX-License-Identifier: Apache-2.0